In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
os.chdir("D:/data/")
os.getcwd()

'D:\\data'

In [2]:
# 1列hadm_id,58列SAPS-II特征，1列年龄，1列性别，4列死亡标签，共65列
df=pd.read_csv("feature_clean.csv")
del df['Unnamed: 0']
df[:2]

,hadm_id,51,184,190,211,442,454,455,676,678,...,226567,226584,227488,227489,age,gender,label1-2d,label2-5d,label3-10d,label4-30d
0,100001,117.786058,4,0.582271,99.312406,119.392324,6,118.497302,36.831826,98.167991,...,542.34375,1231.078431,7776.896552,8641.809524,35,0,0,0,0,0
1,100003,117.786058,4,0.582271,99.312406,119.392324,6,118.497302,36.831826,98.167991,...,542.34375,1231.078431,7776.896552,8641.809524,59,1,0,0,0,0


In [3]:
df.shape

(47642, 65)

In [4]:
len(df.hadm_id.unique())

47642

In [5]:
# 1列hadm_id,58列SAPS-II特征，1列年龄，1列性别，4列死亡标签，共65列
#分别取出数据集中的自变量和因变量
#先用患者入院后第2天的死亡率作为预测变量
X=df.iloc[:,1:61]
y=df.iloc[:,62]
XX=X.values
yy=y.values
#记录数据框的列名
col_name=pd.concat([X,y],axis=1).columns

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.metrics import confusion_matrix
from statistics import mean
from imblearn.combine import SMOTEENN 
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

def kfold_imb( XX, yy, model):
    cv = StratifiedKFold(n_splits=5,shuffle=False,random_state=None)
    
    acc_total=[]
    prec_total=[]
    reca_total=[]
    f1_total=[]
    auc_total=[]
    prc_total=[]
    ks_total=[]
    for train, test in cv.split(XX,yy):
        #过采样
        smote_enn = SMOTEENN(random_state=1)
        X_resampled, y_resampled = smote_enn.fit_sample(XX[train],yy[train])
        model.fit(X_resampled, y_resampled)
        score = model.fit(X_resampled, y_resampled).predict_proba(XX[test])
        preds = model.predict(XX[test])
        
        from sklearn.metrics import f1_score
        acc=accuracy_score(yy[test], preds)
        prec=precision_score(yy[test],preds)
        reca=recall_score(yy[test], preds)
        f1=f1_score(yy[test], preds)
        auc=roc_auc_score(yy[test],score[:,1])
        prc= average_precision_score(yy[test], score[:,1])
        fpr, tpr, thresholds = roc_curve(yy[test], score[:, 1])
        ks_value=max(abs(fpr-tpr))
        
        acc_total.append(acc)
        prec_total.append(prec)
        reca_total.append(reca)
        f1_total.append(f1)
        auc_total.append(auc)
        prc_total.append(prc)
        ks_total.append(ks_value)
    accuracy=np.mean(acc_total)
    precision=np.mean(prec_total)
    recall=np.mean(reca_total)
    f1_score=np.mean(f1_total)
    auroc=np.mean(auc_total)
    auprc=np.mean(prc_total)
    ks=np.mean(ks_total)
        #score = evaluation(yy[test], y_predict)
        #score_total.append(score)
    #score = np.mean(score_total)
    mm=pd.DataFrame(data=[accuracy, precision,recall,f1_score,auroc,auprc,ks],
                    index=["accuracy", "precision", "recall", "f1_score","roc_auc_score","prc_auc_score","ks-value"])
    return mm

def kfold( XX, yy, model):
    cv = StratifiedKFold(n_splits=5,shuffle=False,random_state=None)
    #kf = KFold(n_splits=4)
    acc_total=[]
    prec_total=[]
    reca_total=[]
    f1_total=[]
    auc_total=[]
    prc_total=[]
    ks_total=[]
    for train, test in cv.split(XX,yy):
        model.fit(XX[train], yy[train])
        preds = model.predict(XX[test])
        score = model.fit(XX[train], yy[train]).predict_proba(XX[test])
        from sklearn.metrics import f1_score
        acc=accuracy_score(yy[test], preds)
        prec=precision_score(yy[test],preds)
        reca=recall_score(yy[test], preds)
        f1=f1_score(yy[test], preds)
        #auc=roc_auc_score(yy[test], preds)
        ####修改后
        auc=roc_auc_score(yy[test], score[:,1])
        prc= average_precision_score(yy[test], score[:,1])
        fpr, tpr, thresholds = roc_curve(yy[test], score[:, 1])
        ks_value=max(abs(fpr-tpr))
        
        acc_total.append(acc)
        prec_total.append(prec)
        reca_total.append(reca)
        f1_total.append(f1)
        auc_total.append(auc)
        prc_total.append(prc)
        ks_total.append(ks_value)
    
    accuracy=np.mean(acc_total)
    precision=np.mean(prec_total)
    recall=np.mean(reca_total)
    f1_score=np.mean(f1_total)
    auroc=np.mean(auc_total)
    auprc=np.mean(prc_total)
    ks=np.mean(ks_total)
    mm=pd.DataFrame(data=[accuracy, precision,recall,f1_score,auroc,auprc,ks],
                    index=["accuracy", "precision", "recall", "f1_score","roc_auc_score","prc_auc_score","ks-value"])    
    return mm

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

[LogisticRegression(C=1.0,random_state=None),
 DecisionTreeClassifier(criterion='gini',random_state=None),
 RandomForestClassifier(max_depth=None,random_state=None),
 KNeighborsClassifier(n_neighbors=5),
 GradientBoostingClassifier(n_estimators=100,max_features=None,max_depth=3,random_state=None),
 AdaBoostClassifier(n_estimators=50,random_state=None),
 LGBMClassifier(random_state=None),
 XGBClassifier()
]

lr=LogisticRegression(C=1.0,random_state=None)
print("imblance_logistic regression's accuracy:\n{}".format(kfold_imb( XX, yy, lr)))    
print("logistic regression's accuracy:\n{}".format(kfold( XX, yy, lr)))

D:\software\anaconda\path\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
D:\software\anaconda\path\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-r

imblance_logistic regression's accuracy:
                      0
accuracy       0.577789
precision      0.094846
recall         0.745693
f1_score       0.168279
roc_auc_score  0.731144
prc_auc_score  0.172914
ks-value       0.350998


D:\software\anaconda\path\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
D:\software\anaconda\path\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-r

logistic regression's accuracy:
                      0
accuracy       0.942005
precision      0.320430
recall         0.008428
f1_score       0.016236
roc_auc_score  0.719798
prc_auc_score  0.162759
ks-value       0.351147


D:\software\anaconda\path\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [9]:
#decision tree
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier(criterion='gini',random_state=None)
print("imblance_decision tree's accuracy:\n{}".format(kfold_imb( XX, yy, DT)))
print("decision tree's accuracy:\n{}".format(kfold( XX, yy, DT)))

imblance_decision tree's accuracy:
                      0
accuracy       0.863419
precision      0.191837
recall         0.430933
f1_score       0.265451
roc_auc_score  0.660315
prc_auc_score  0.115629
ks-value       0.320631
decision tree's accuracy:
                      0
accuracy       0.913669
precision      0.270595
recall         0.299374
f1_score       0.284056
roc_auc_score  0.625565
prc_auc_score  0.121686
ks-value       0.251131


In [10]:
#random forest
from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier(max_depth=None,random_state=None)
print("imblance_RandomForest's accuracy:\n{}".format(kfold_imb( XX, yy, RF)))
print("RandomForest's accuracy:\n{}".format(kfold( XX, yy, RF)))

imblance_RandomForest's accuracy:
                      0
accuracy       0.919252
precision      0.346607
recall         0.461350
f1_score       0.395653
roc_auc_score  0.860330
prc_auc_score  0.348112
ks-value       0.561353
RandomForest's accuracy:
                      0
accuracy       0.950842
precision      0.837621
recall         0.175893
f1_score       0.290451
roc_auc_score  0.856357
prc_auc_score  0.451003
ks-value       0.563797


In [11]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
KNN=KNeighborsClassifier(n_neighbors=5)
print("imblance_KNN's accuracy:\n{}".format(kfold_imb( XX, yy, KNN)))
print("KNN's accuracy:\n{}".format(kfold( XX, yy, KNN)))

imblance_KNN's accuracy:
                      0
accuracy       0.730595
precision      0.114328
recall         0.548554
f1_score       0.189213
roc_auc_score  0.679298
prc_auc_score  0.107315
ks-value       0.308154
KNN's accuracy:
                      0
accuracy       0.942236
precision      0.459248
recall         0.055698
f1_score       0.099198
roc_auc_score  0.632636
prc_auc_score  0.127253
ks-value       0.249131


In [12]:
#GBDT
from sklearn.ensemble import GradientBoostingClassifier
GBDT=GradientBoostingClassifier(n_estimators=100,max_features=None,max_depth=3,random_state=None)
print("imblance_GBDT's accuracy:\n{}".format(kfold_imb( XX, yy, GBDT)))
print("GBDT's accuracy:\n{}".format(kfold( XX, yy, GBDT)))

imblance_GBDT's accuracy:
                      0
accuracy       0.876643
precision      0.245724
recall         0.557353
f1_score       0.340990
roc_auc_score  0.842870
prc_auc_score  0.308504
ks-value       0.529764
GBDT's accuracy:
                      0
accuracy       0.950065
precision      0.733841
recall         0.201182
f1_score       0.315552
roc_auc_score  0.863361
prc_auc_score  0.438068
ks-value       0.571880


In [19]:
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier
Ada=AdaBoostClassifier(n_estimators=50,random_state=None)
print("imblance_AdaBoost's accuracy:\n{}".format(kfold_imb( XX, yy, Ada)))
print("AdaBoost's accuracy:\n{}".format(kfold( XX, yy, Ada)))

imblance_AdaBoost's accuracy:
                      0
accuracy       0.846396
precision      0.198744
recall         0.554430
f1_score       0.292543
roc_auc_score  0.813264
prc_auc_score  0.251393
ks-value       0.476715
AdaBoost's accuracy:
                      0
accuracy       0.946770
precision      0.597346
recall         0.221698
f1_score       0.322861
roc_auc_score  0.851931
prc_auc_score  0.386605
ks-value       0.546180


In [20]:
#lightGBM
import lightgbm.LGBMClassifier as LGBMClassifier
lgb=LGBMClassifier(random_state=None)
print("imblance_lightgbm's accuracy:\n{}".format(kfold_imb( XX, yy, lgb)))
print("lightgbm's accuracy:\n{}".format(kfold( XX, yy, lgb)))

imblance_lightgbm's accuracy:
                      0
accuracy       0.915369
precision      0.337636
recall         0.495057
f1_score       0.401355
roc_auc_score  0.859476
prc_auc_score  0.378765
ks-value       0.552998
lightgbm's accuracy:
                      0
accuracy       0.951262
precision      0.706267
recall         0.256514
f1_score       0.375816
roc_auc_score  0.877147
prc_auc_score  0.465020
ks-value       0.594606


In [21]:
#xgboost
from xgboost import XGBClassifier
xgb=XGBClassifier()
print("imblance_xgboost's accuracy:\n{}".format(kfold_imb( XX, yy, xgb)))
print("xgboost's accuracy:\n{}".format(kfold( XX, yy, xgb)))

D:\software\anaconda\path\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:10:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:10:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\software\anaconda\path\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:11:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:11:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\software\anaconda\path\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[18:16:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:16:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\software\anaconda\path\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:16:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:16:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\software\anaconda\path\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:16:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:16:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgboost's accuracy:
                      0
accuracy       0.949960
precision      0.657598
recall         0.264572
f1_score       0.377057
roc_auc_score  0.867586
prc_auc_score  0.446422
ks-value       0.566223
